In this notebook, we will create and compare several models. Our target feature for this notebooks is (Likes+Comments/(Views+1))
which we will call "engagement". We have added 1 to the denonimator just to ensure we never divide by zero. 

Please note that we are not really creating a "model" of our data. That is, we do not believe our data is fully explained by our features nor that there is underlying "true" relationship that is linear. We are not using linear regression to predict the data or model the data. Instead, we are finding the line of best fit and using it as an indication of trend. 

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"all_features_final.csv")
features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref", "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products", "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]

#Quickly making the "hashtags" column categorical 
df["hashtags"] = 1 * df["hashtags"].astype(bool)

#Create the target column $y$ here 

df["y"] = (df["likes"] + df["commentsCount"])  / (df["viewCount"] + 1) 

#get rid of noisy columns
df = df[ features + ["y"] ] 

In [3]:
#import everything

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


In [4]:
#Do an 80-20 Train Test Split Here. Never ever touch the testing set please!

df_train, df_test = train_test_split(df, shuffle = True, test_size = .2, random_state = 42) #We can't stratify because we have too many categorical features. I hope this is ok
#DO NOT TOUCH THE ABOVE X_TEST VARIABLE FOR ANY REASON

#We want a very basic idea of the MSE and other metrics for each model, before we do proper cross-validation. We use a secondary split for this.
df_tt, df_ho = train_test_split(df_train, shuffle = True, test_size = .2, random_state = 42)


In [5]:
#Create the baseline model here

class BaseMeanModel():
    def __init__(self):
        self.mean_value = None
    
    def fit(self, values : pd.Series):
        self.mean_value = values.mean()

    def predict(self, inputs=None):
        if inputs is None:
            return self.mean_value
        return len(inputs) * [self.mean_value]
    
    
model = BaseMeanModel()
model.fit(df_tt["y"])

# R2 is negative because training set and the hold out set have different average values
y_pred = model.predict(df_ho[features])
rmse = root_mean_squared_error(df_ho["y"], y_pred)
r2 = r2_score(df_ho["y"], y_pred)
print(f"Root Mean Squared Error: {rmse:.6f}")
print(f"R-squared: {r2:.4f}")

Root Mean Squared Error: 0.037166
R-squared: -0.0000


In [6]:
#Creat the basic linear regression model here 
###Fitting the basic linear regression model using the training set and print the summary of the model.
model = LinearRegression()
model.fit(df_tt[features], df_tt["y"])
# Evaluate the model
y_pred = model.predict(df_ho[features])
rmse = root_mean_squared_error(df_ho["y"], y_pred)
r2 = r2_score(df_ho["y"], y_pred)
print(f"Root Mean Squared Error: {rmse:.6f}")
print(f"R-squared: {r2:.4f}")

Root Mean Squared Error: 0.036475
R-squared: 0.0368


In [7]:
#Create the basic linear regression model here with lasso regression. 


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
from sklearn.model_selection import cross_val_score


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_tt[features] )
X_test_scaled = scaler.transform( df_ho[features])

alpha = 0.0001
lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)

#cv_scores = cross_val_score(lasso, X_train_scaled, df_tt["y"], cv=5,scoring='neg_mean_squared_error')
#cv_rmse = np.sqrt(-cv_scores.mean())

lasso.fit(X_train_scaled, df_tt["y"])

y_pred = lasso.predict(X_test_scaled)

# Calculate metrics
mse = mean_squared_error(df_ho["y"], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(df_ho["y"], y_pred)
r2 = r2_score(df_ho["y"], y_pred)
exp_var = explained_variance_score(df_ho["y"], y_pred)


print(f"\nAlpha: {alpha}")
print(f"Test MSE: {mse:.6f}, RMSE: {rmse:.6f}, MAE: {mae:.6f}")
print(f"R² Score: {r2:.6f}, Explained Variance: {exp_var:.6f}")
#print(f"Cross-validated RMSE: {cv_rmse:.6f}")



Alpha: 0.0001
Test MSE: 0.001330, RMSE: 0.036467, MAE: 0.023660
R² Score: 0.037211, Explained Variance: 0.037232


In [8]:
#Create a model whose features include all interaction terms
pipe = Pipeline([  ("interaction terms", PolynomialFeatures(degree = 2, interaction_only = True, include_bias = False) ),
                   ("linear model", LinearRegression())
]) 
#setting degree = 2 creates all pairwise interaction terms. 

pipe.fit( df_tt[features], df_tt["y"]) 
pred = pipe.predict( df_ho[features] )

mse = mean_squared_error(df_ho["y"], pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(df_ho["y"], pred)
r2 = r2_score(df_ho["y"], pred)
exp_var = explained_variance_score(df_ho["y"], pred)


print(f"Test MSE: {mse:.6f}, RMSE: {rmse:.6f}, MAE: {mae:.6f}")
print(f"R² Score: {r2:.6f}, Explained Variance: {exp_var:.6f}")

Test MSE: 0.001289, RMSE: 0.035906, MAE: 0.023126
R² Score: 0.066610, Explained Variance: 0.066661


In [9]:
#Create a model with all interaction terms and lasso regression 

#Create a model with all interaction terms and lasso regression 
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df_tt = scaler.fit_transform( df_tt[features] )
scaled_df_ho = scaler.transform( df_ho[features] )

# using lasso cv to find the best alpha
# lasso = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=np.logspace(-4, 1, 30))
# lasso.fit(df_tt[features], df_tt["y"])
# pred = lasso.predict(df_ho[features])
# print("Lasso CV MSE:", root_mean_squared_error(df_ho["y"], pred))
# print("Optimal alpha:", lasso.alpha_)

pipe = Pipeline([
    ("interaction terms", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ("lasso", Lasso(alpha=0.0001, max_iter=10000))
])
pipe.fit(scaled_df_tt, df_tt["y"])
pred = pipe.predict(scaled_df_ho)
print("MSE : ", root_mean_squared_error(df_ho["y"], pred))

# Get lasso coefficients
lasso_coeffs = pd.Series(pipe.named_steps['lasso'].coef_, index=pipe.named_steps['interaction terms'].get_feature_names_out(features))
lasso_coeffs = lasso_coeffs[lasso_coeffs != 0]
# print(lasso_coeffs)

MSE :  0.03591451395194999


In [10]:
#Do cross-validation to compare all models 

#Model 0: Baseline Average
#Model 1: Basic Linear Regression Model
#Model 2: Linear Regression with Lasso
#Model 3: Linear Regression with interaction terms
#Model 4: Linear Regression with interactions and lasso

from sklearn.model_selection import KFold
num_splits = 5
num_models = 5

kfold = KFold(num_splits,
              random_state = 42,
              shuffle=True)

rmses = np.zeros((num_models, num_splits))

for i, (train_index, test_index) in enumerate(kfold.split(df_train)): 

    df_tt = df_train.iloc[train_index]
    df_ho = df_train.iloc[test_index] 

    #Model 0: Baseline Average
    model = BaseMeanModel()
    model.fit(df_tt["y"])
    y_pred = model.predict(df_ho[features])
    rmses[0,i] = root_mean_squared_error(df_ho["y"], y_pred)

    #Model 1: Basic Linear Regression Model
    model = LinearRegression()
    model.fit(df_tt[features], df_tt["y"])
    y_pred = model.predict(df_ho[features])
    rmses[1,i] = root_mean_squared_error(df_ho["y"], y_pred)

    #Model 2: Linear Regression with Lasso
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(df_tt[features] )
    X_test_scaled = scaler.transform( df_ho[features])
    alpha = 0.0001
    lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)
    lasso.fit(X_train_scaled, df_tt["y"])
    y_pred = lasso.predict(X_test_scaled)
    rmses[2,i] = root_mean_squared_error(df_ho["y"], y_pred)

    #Model 3: Linear Regression with interaction terms
    pipe = Pipeline([  ("interaction terms", PolynomialFeatures(degree = 2, interaction_only = True, include_bias = False) ),
                   ("linear model", LinearRegression())
                    ]) 
    pipe.fit( df_tt[features], df_tt["y"]) 
    y_pred = pipe.predict( df_ho[features] )
    rmses[3,i] = root_mean_squared_error(df_ho["y"], y_pred)

    #Model 4: Linear Regression with interactions and lasso
    scaler = StandardScaler()
    scaled_df_tt = scaler.fit_transform( df_tt[features] )
    scaled_df_ho = scaler.transform( df_ho[features] )
    pipe = Pipeline([
    ("interaction terms", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ("lasso", Lasso(alpha=0.0001, max_iter=10000))
                    ])
    pipe.fit(scaled_df_tt, df_tt["y"])
    y_pred = pipe.predict(scaled_df_ho)
    rmses[4,i] = root_mean_squared_error(df_ho["y"], y_pred)


print(rmses)

[[0.03716583 0.03233239 0.03303423 0.03168026 0.03176396]
 [0.03647535 0.03170704 0.03215983 0.03037636 0.03049902]
 [0.03646742 0.0316984  0.03216117 0.03038644 0.03051464]
 [0.03590635 0.03134345 0.03197815 0.03012346 0.03014119]
 [0.03591451 0.03126992 0.03192117 0.03004606 0.03010355]]


In [11]:
rmses.mean(axis = 1)
#Pretty much as expected, models with the most features (i.e., models that included interaction terms) did the best.
#Interestingly, Lasso regression slightly outperformed regular linear regression. 

array([0.03319534, 0.03224352, 0.03224562, 0.03189852, 0.03185104])

In [12]:
#Note that in the RMSE matrix, each ROW is a different model and each COLUMN is a CV. 
#In the above we took an average of each ROW. 
#As a sanity check we look at the full RMSE matrix and see that yes, in each column the first row is the largest number,
#the second row is (often) the second largest number, etc. This means there is a consistent pattern as to the performance of our models
#and it is not just random noise. 

---Final Interpretation---

Almost as expected, the model with the most features (i.e., the model that includes all interaction terms) performed the best. This could just be because every time a linear model has more features it will perform better, in the sense that it will have a lower rmse. 

Interestingly, our lasso model slightly outperformed our non-lasso model. 

We want to take the coefficients of our highest performing model and compare them. This is where our analysis was really headed.
Mathematically, the features with the highest coefficients are those that contribute the most to the trend line.



In [14]:
#A final fitting to the whole data set 
scaler = StandardScaler()
scaled_df = scaler.fit_transform( df[features] )
pipe = Pipeline([
    ("interaction terms", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ("lasso", Lasso(alpha=0.0001, max_iter=10000))
                    ])
pipe.fit(scaled_df, df["y"])

lasso_coeffs = pd.Series(pipe.named_steps['lasso'].coef_, index=pipe.named_steps['interaction terms'].get_feature_names_out(features))
sig_lasso_coeffs = lasso_coeffs[lasso_coeffs != 0]
print(sig_lasso_coeffs)

popular_brand              -0.002025
has_any_affiliate           0.001928
product                    -0.002123
budget                     -0.000927
self_ref                    0.002798
                              ...   
prime_hour hashtags         0.000056
prime_hour hasAdinTitle    -0.000461
prime_hour hasAdinText      0.000640
hashtags hasAdinText       -0.000285
hasAdinTitle hasAdinText    0.000018
Length: 93, dtype: float64


In [15]:
new = sig_lasso_coeffs.sort_values(key=abs)

In [16]:
new.tail(10)

self_ref hashtags           0.001442
prime_hour                  0.001700
korean                      0.001704
skills/teach hasAdinText    0.001752
has_any_affiliate           0.001928
popular_brand              -0.002025
product                    -0.002123
self_ref                    0.002798
skincare                   -0.003003
hashtags                   -0.005041
dtype: float64